In [ ]:
#TASK 2 : Restaurant Recommendation

# Importing Libraries 
import pandas as pd

class RestaurantRecommender:
    def __init__(self, csv_path):
        self.datafile = pd.read_csv('/content/Dataset .csv')
        self._preprocess_data()
        
    def _preprocess_data(self):
        self.datafile = self.datafile.dropna(subset=['Cuisines', 'Price range'])
        self.datafile['Aggregate rating'].fillna(0, inplace=True)
        self.datafile['Votes'].fillna(0, inplace=True)
    
    def recommend_restaurants(self, preferred_cuisine=None, max_price=None, 
                             min_rating=None, city=None, locality=None, 
                             sort_by='Votes', top_n=5):
        """
        Recommend restaurants based on user preferences
        
        Args:
            preferred_cuisine (str): Preferred cuisine type
            max_price (int): Maximum price range (1-4)
            min_rating (float): Minimum aggregate rating (0-5)
            city (str): Preferred city
            locality (str): Preferred locality
            sort_by (str): Sort results by 'Votes' or 'Aggregate rating'
            top_n (int): Number of recommendations to return
        """
        filtered = self.datafile.copy()
        active_filters = []
        
        # Apply filters
        if max_price is not None:
            filtered = filtered[filtered['Price range'] <= max_price]
            active_filters.append(f"Price ≤ {max_price}")
        
        if preferred_cuisine is not None:
            filtered = filtered[filtered['Cuisines'].str.contains(
                preferred_cuisine, case=False, na=False)]
            active_filters.append(f"Cuisine: {preferred_cuisine}")
        
        if min_rating is not None:
            filtered = filtered[filtered['Aggregate rating'] >= min_rating]
            active_filters.append(f"Rating ≥ {min_rating}")
        
        if city is not None:
            filtered = filtered[filtered['City'].str.contains(
                city, case=False, na=False)]
            active_filters.append(f"City: {city}")
        
        if locality is not None:
            filtered = filtered[filtered['Locality'].str.contains(
                locality, case=False, na=False)]
            active_filters.append(f"Locality: {locality}")
        
        if len(filtered) == 0:
            print("No restaurants found matching your preferences.\n")
            return None
        
        # Sort and return top N results
        if sort_by in filtered.columns:
            filtered = filtered.sort_values(sort_by, ascending=False)
        
        result = filtered.head(top_n)
        
        # Display filters used
        if active_filters:
            print(f"Filters Applied: {', '.join(active_filters)}")
        print(f"Sorted by: {sort_by}")
        print(f"\nTop {len(result)} Recommendations:\n")
        for i, (idx, row) in enumerate(result.iterrows(), start=1):
            print(f"{i}. {row['Restaurant Name']}")
            print(f"   Cuisine: {row['Cuisines']}")
            print(f"   Price Range: {row['Price range']}/4")
            print(f"   Rating: {row['Aggregate rating']}/5  |  Votes: {row['Votes']}")
            print(f"   Location: {row['Locality']}, {row['City']}\n")
        
        return result[['Restaurant Name', 'Cuisines', 'Price range', 
                      'Aggregate rating', 'Votes', 'City', 'Locality']]


# Example Usage and Testing
if __name__ == "__main__":
    # Initialize the recommender system
    recommender = RestaurantRecommender('/content/Dataset .csv')
    
    # Test Case 1: Budget-friendly Japanese restaurants
    print("Test Case 1: Budget-Friendly Japanese Food")
    print("-" * 60)
    recommender.recommend_restaurants(
        preferred_cuisine='Japanese',
        max_price=3,
        min_rating=3.5,
        sort_by='Aggregate rating',
        top_n=5
    )
    
    # Test Case 2: High-rated Italian restaurants
    print("\nTest Case 2: High-Rated Italian Restaurants")
    print("-" * 60)
    recommender.recommend_restaurants(
        preferred_cuisine='Italian',
        min_rating=4.0,
        sort_by='Votes',
        top_n=5
    )